In [135]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import log_loss
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, fixed

In [7]:
df = pd.read_csv('../nss_capstone/df_rebuild.csv').drop(columns = 'Unnamed: 0').fillna('')

In [11]:
#df_val_21_22 = df.loc[(df['Date'] <= '2022-06-16') & (df['Date'] >= '2021-10-19')]

In [22]:
# df_train_07_20 = df.drop(df.index[0:2646])

In [23]:
# df_train_07_20.to_csv('../nss_capstone/df_train_07_20.csv')

In [13]:
#df_val_21_22.to_csv('../nss_capstone/df_val_21_22.csv')

In [19]:
# df.loc[(df['Date'] <= '2021-07-01') & (df['Date'] >= '2007-10-30')]

In [39]:
# df_train[['ML','ML_pct','Open Total', 'Close Total', 'Open Spread', 'Close Spread']] = df_train[['ML','ML_pct','Open Total', 'Close Total', 'Open Spread', 'Close Spread']].apply(pd.to_numeric, errors='coerce')

In [40]:
# df_train['ML_pct'] = df_train['ML_pct'] / 100

In [89]:
df_train = pd.read_csv('../nss_capstone/df_train_07_20.csv').drop(columns = 'Unnamed: 0').fillna('')
df_val = pd.read_csv('../nss_capstone/df_val_21_22.csv').drop(columns = 'Unnamed: 0').fillna('')

In [122]:
df_train['ML'] = df_train['ML'].astype(str)
df_train['ML_pct'] = df_train['ML_pct'].astype(str)
df_train['Open Spread'] = df_train['Open Spread'].astype(str)
df_train['Close Spread'] = df_train['Close Spread'].astype(str)
df_train['Open Total'] = df_train['Open Total'].astype(str)
df_train['Close Total'] = df_train['Close Total'].astype(str)

In [123]:
df_val['ML'] = df_val['ML'].astype(str)
df_val['ML_pct'] = df_val['ML_pct'].astype(str)
df_val['Open Spread'] = df_val['Open Spread'].astype(str)
df_val['Close Spread'] = df_val['Close Spread'].astype(str)
df_val['Open Total'] = df_val['Open Total'].astype(str)
df_val['Close Total'] = df_val['Close Total'].astype(str)

In [124]:
# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the Color column
df_train['ML'] = le.fit_transform(df_train['ML'])

df_train['ML_pct'] = le.fit_transform(df_train['ML_pct'])
df_train['Open Spread'] = le.fit_transform(df_train['Open Spread'])
df_train['Close Spread'] = le.fit_transform(df_train['Close Spread'])
df_train['Open Total'] = le.fit_transform(df_train['Open Total'])
df_train['Close Total'] = le.fit_transform(df_train['Close Total'])


In [125]:
# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the Color column
df_val['ML'] = le.fit_transform(df_val['ML'])

df_val['ML_pct'] = le.fit_transform(df_val['ML_pct'])
df_val['Open Spread'] = le.fit_transform(df_val['Open Spread'])
df_val['Close Spread'] = le.fit_transform(df_val['Close Spread'])
df_val['Open Total'] = le.fit_transform(df_val['Open Total'])
df_val['Close Total'] = le.fit_transform(df_val['Close Total'])

In [126]:
df_val.info()#.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2236 entries, 0 to 2235
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_id       2236 non-null   int64 
 1   Date          2236 non-null   object
 2   VH            2236 non-null   object
 3   Team          2236 non-null   object
 4   Final         2236 non-null   int64 
 5   ML            2236 non-null   int32 
 6   ML_pct        2236 non-null   int32 
 7   Open Total    2236 non-null   int32 
 8   Close Total   2236 non-null   int32 
 9   Open Spread   2236 non-null   int32 
 10  Close Spread  2236 non-null   int32 
 11  W/L_fav       2236 non-null   int64 
 12  W/L_dog       2236 non-null   int64 
 13  ATS_fav       2236 non-null   int64 
 14  W/L_H_fav     2236 non-null   int64 
 15  W/L_V_fav     2236 non-null   int64 
 16  W/L_H_dog     2236 non-null   int64 
 17  W/L_V_dog     2236 non-null   int64 
 18  ATS_H_fav     2236 non-null   int64 
 19  ATS_V_

            *******************Gradient Boost***************

In [127]:
variables = ['ML', 'Close Total', 'Close Spread']#'Open Total','Open Spread', 

# Subset your DataFrame with the selected variables
X = df_train[variables]
y = df_train['W/L_bool']

# Group by 'game_id'
grouped = df.groupby('game_id')

# Initialize train and test indices
train_indices, test_indices = [], []

# Iterate through groups and split indices
for _, group_indices in grouped.groups.items():
    group_indices = [index for index in group_indices if index in df.index]  # Filter out invalid indices
    group_train_indices, group_test_indices = train_test_split(group_indices, test_size=0.2, random_state=321)
    train_indices.extend(group_train_indices)
    test_indices.extend(group_test_indices)

# Select the corresponding rows from X and y for training and testing
X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]

# Fit the GradientBoostingClassifier
gbr = GradientBoostingClassifier(n_estimators=1000).fit(X_train, y_train)

# Evaluate the model
print(classification_report(y_test, gbr.predict(X_test)))
print("Accuracy:", accuracy_score(y_test, gbr.predict(X_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, gbr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.21      0.32     11463
           1       0.32      0.80      0.45      5201

    accuracy                           0.39     16664
   macro avg       0.51      0.51      0.39     16664
weighted avg       0.58      0.39      0.36     16664

Accuracy: 0.3931829092654825
Confusion Matrix:
[[2367 9096]
 [1016 4185]]


In [128]:
X_val = df_val[variables]
y_val = df_val['W/L_bool']

y_pred_val = gbr.predict(X_val)


print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))
# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

              precision    recall  f1-score   support

           0       0.44      0.12      0.19      1118
           1       0.49      0.85      0.62      1118

    accuracy                           0.48      2236
   macro avg       0.47      0.48      0.41      2236
weighted avg       0.47      0.48      0.41      2236

[[136 982]
 [172 946]]


In [132]:
variables = ['ML', 'Close Total', 'Close Spread']#'Open Total','Open Spread', 

# Subset your DataFrame with the selected variables
X = df_train[variables]
y = df_train['W/L_bool']

# Group by 'game_id'
grouped = df.groupby('game_id')

# Initialize train and test indices
train_indices, test_indices = [], []

# Iterate through groups and split indices
for _, group_indices in grouped.groups.items():
    group_indices = [index for index in group_indices if index in df.index]  # Filter out invalid indices
    group_train_indices, group_test_indices = train_test_split(group_indices, test_size=0.2, random_state=321)
    train_indices.extend(group_train_indices)
    test_indices.extend(group_test_indices)

# Select the corresponding rows from X and y for training and testing
X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]

# Fit the GradientBoostingClassifier
#gbr = GradientBoostingClassifier(n_estimators=1000).fit(X_train, y_train)
#rf = RandomForestClassifier(n_estimators=1000)
#rf.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=1000)
xgb_model.fit(X_train, y_train)



# Evaluate the model
print(classification_report(y_test, gbr.predict(X_test)))
print("Accuracy:", accuracy_score(y_test, gbr.predict(X_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, gbr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.21      0.32     11463
           1       0.32      0.80      0.45      5201

    accuracy                           0.39     16664
   macro avg       0.51      0.51      0.39     16664
weighted avg       0.58      0.39      0.36     16664

Accuracy: 0.3931829092654825
Confusion Matrix:
[[2367 9096]
 [1016 4185]]


In [133]:
X_val = df_val[variables]
y_val = df_val['W/L_bool']

#y_pred_val = grb.predict(X_val)
#y_pred_val = rf.predict(X_val)

y_pred_val = xgb_model.predict(X_val)


print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))
# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

              precision    recall  f1-score   support

           0       0.42      0.14      0.20      1118
           1       0.48      0.81      0.61      1118

    accuracy                           0.47      2236
   macro avg       0.45      0.47      0.41      2236
weighted avg       0.45      0.47      0.41      2236

[[152 966]
 [213 905]]


In [ ]:
df = pd.read_csv('../nss_capstone/df.csv').drop(columns = 'Unnamed: 0').fillna('')


variables = ['ML',
             'Open Total',
             'Open Spread',
            'Close Total',
            'Close Spread']


X = df[variables]
y = df['ATS']


grouped = df.groupby('game_id')
train_indices, test_indices = [], []
for _, group_indices in grouped.groups.items():
    group_train_indices, group_test_indices = train_test_split(group_indices, test_size=0.2, random_state=321)
    train_indices.extend(group_train_indices)
    test_indices.extend(group_test_indices)


X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]


xgb_model = xgb.XGBClassifier(n_estimators=1000)
xgb_model.fit(X_train, y_train)

y_pred_test = xgb_model.predict(X_test)


print(classification_report(y_test, y_pred_test))
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

In [136]:
def create_histogram(ML_pct):
    filtered_df = df_train[df_train['ML_pct'] == ML_pct]
    game_counts = filtered_df.shape[0] // 2  # Divide the number of rows by 2 to get the count of games
    
    win_counts = filtered_df[filtered_df['W/L_bool'] == 1].shape[0]  // 2# Divide the count by 2 for games
    loss_counts = filtered_df[filtered_df['W/L_bool'] == 0].shape[0] //2  # Divide the count by 2 for games
    
#     if game_counts == 0:
#         win_ratio = 0
#         loss_ratio = 0
#     else:
    win_ratio = win_counts / game_counts
    loss_ratio = loss_counts / game_counts

    labels = ['Favorite Wins', 'Underdog Wins']
    counts = [win_counts, loss_counts]
    
    plt.figure(figsize=(8, 6))
    plt.barh(labels, counts)
    plt.xlabel('Count')
    plt.ylabel('W/L_bool')
    plt.title(f'Favorite and Underdog Win Ratios Per Closing Moneyline % = {ML_pct}')
    plt.text(max(counts), 0, f'{win_ratio:.2f}', ha='left', va='center')
    plt.text(max(counts), 1, f'{loss_ratio:.2f}', ha='left', va='center')
    plt.show()

interact(create_histogram, ML_pct=widgets.FloatSlider(min=0.5, max=0.99, step=0.01, value=0.5))

interactive(children=(FloatSlider(value=0.5, description='ML_pct', max=0.99, min=0.5, step=0.01), Output()), _…

<function __main__.create_histogram(ML_pct)>